In [1]:
import src.dataset as ds
import numpy as np
from src.embeddings import load_pretrained_embedding
from keras.layers import Embedding
from keras.initializers import Constant
from keras import Sequential

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\Eli\Anaconda3\envs\dlinto\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
X, y, tokenizer = ds.load_tokenized_data()

100%|██████████████████████████████████████████████████████████████████| 615/615 [00:01<00:00, 359.77it/s]


In [3]:
embedding_matrix = load_pretrained_embedding()

In [5]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
EMBEDDING_DIM = 300
num_words = len(tokenizer.word_index) + 1
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=1,
                            trainable=False)

In [ ]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(CuDNNLSTM(50)))
model.add(Dense(num_words, activation='softmax'))
model.summary()

In [ ]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=5, verbose=2)

In [ ]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = np.array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
    		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result